<a href="https://colab.research.google.com/github/marichaf/credential-attainment-wfs/blob/main/credential_attainment_3_24_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Credential attainment for WIOA Formula
# Maricha, created 10/8/24

# TODO:
    # Upload outcomes report, just for cohort timeframe, to check for employment for those who completed alt/GED/HSD (not just youth, anyone)
    # Upload enrollment report (for as far back as the same date of the ITS report) to check co-enrollments - CA counts towards all programs there are in

In [ ]:
# This block is required for use on Google Drive only. DO NOT EDIT

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # probably don't need this here
from datetime import datetime
from tabulate import tabulate

## Step 1: Load ETO Data reports
There are 2 different ETO reports you can use, both are under Local Reporter Data:
1) `Individualized Training and Support Services TPs`
2) `Local Reporter Services Report OPTIMIZED`
There are minor differences depending on which you use, and they are noted in the script. These are mainly different field names you'll need to adjust depending on which you use. Local reporter services also has ALL services, not just ITSS. However, we are only looking at ITSS touchpoints here and filtering down to eligible training serices. There may be a scenario in which you might want to look at other touchpoints that were mistakenly used instead of training services (?).

When pulling the report from ETO, go back several years earlier than the start of the cohort/reporting period. For example, if the start date of the cohort period is 7/1/22, pull the report from 7/1/17. This is because a credential attainment counts for participant regardless of when it happened during their participation, but they will be counted only for ther period that contains the participant's exit date.

For example, a participant completed their GED in 2021 but remained enrolled in the program to receive additional support gaining employment (or maybe additional training), and they were exited in April of 2023. They would count in the credential attainment reporting period for 7/1/22 - 6/30/23 (PY24 Q2) even though the date of their actual credential attainment was before the reporting period.

Finally, save the report with filename that reflects the date ranges you pulled. For example:
`ITS Services_7_1_2017 to 6_3_2023.csv`
This helps keep track of the reports we pull and track any changes if needed.

**Important: Export it as a csv from ETO, NOT Excel.*


In [ ]:
### THIS IS THE ONLY CELL THAT REQUIRES EDITING TO RUN THE SCRPT! ###

# Read the CSV file into a pandas dataframe
shared_drive_path = '/content/drive/Shared drives/Service Delivery/ETO Reports/Python Scripts/Credential Attainment/' # Only update if the filepath has changed

# INSERT FILE NAMES HERE: Keep apostrophes and .csv. Make sure it matches exactly.
program_enrollments = 'PER_1_2021 to 12_2021.csv'
itss_report = 'ITSS_1_2017 to 12_2021.csv'
outcomes_report = 'Outcomes_1_2021 to 12_2021.csv'


# UPDATE start and end date of the cohort/reporting period (per the PIRL reporting timeline - or DOL reporting?)
# We need this to check for valid ITAs and Credential Attainments later.
period_start_date = '2021-01-01' # UPDATE: format needs to be yyyy-mm-dd, and keep the apostrophes
period_end_date = '2021-12-31' # UPDATE: format needs to be yyyy-mm-dd, and keep the apostrophes

# UPDATE report date and cohort quarter date - just for filenames.
# This will be pulled in to the filename that gets exported, so the format matters but won't break anything if you change it (it can be different if we want)
report_date = '3_24_25'  # UPDATE to the date you are running this/the date of the data you pull.
quarter = 'PY22' # Decide if we want to use dates or quarters on output file names

In [ ]:
file_path_itss = f'{shared_drive_path}{itss_report}'
itss = pd.read_csv(file_path_itss, low_memory=False, encoding='utf-8', encoding_errors='replace')
print(f'ITSS report has {itss.shape[0]} rows and {itss.shape[1]} columns')


file_path_enrollments = f'{shared_drive_path}{program_enrollments}'
enrollments = pd.read_csv(file_path_enrollments, low_memory=False, encoding='utf-8', encoding_errors='replace')
print(f'Program Enrollments report has {enrollments.shape[0]} rows and {enrollments.shape[1]} columns')

file_path_outcomes = f'{shared_drive_path}{outcomes_report}'
outcomes = pd.read_csv(file_path_outcomes, low_memory=False, encoding='utf-8', encoding_errors='replace')
print(f'Enrollments report has {outcomes.shape[0]} rows and {outcomes.shape[1]} columns')

cohort_start_date = pd.Timestamp(period_start_date)
cohort_end_date = pd.Timestamp(period_end_date)

ITSS report has 20700 rows and 38 columns
Program Enrollments report has 5827 rows and 14 columns
Enrollments report has 1479 rows and 45 columns


In [ ]:
enrollments.columns = enrollments.columns.str.replace(r'[^\x00-\x7F]+', ' ')  # Replace non-ASCII characters with space
enrollments.columns = enrollments.columns.str.strip()  # Strip leading/trailing whitespace

In [ ]:
# From Process doc:
# "Delete all individuals who exited prior to the start date and after the end date of the Cohort Period."
# So we're only looking at people who exited in this time frame

# Convert 'Program Enrollment Start Date' and 'Program Enrollment End Date' to datetime
itss['Program Enrollment Start Date'] = pd.to_datetime(itss['Program Enrollment Start Date'])

# ************
itss['Program Enrollment End Date'] = pd.to_datetime(itss['Program Enrollment End Date'])

#NOTE - if I bring in Program Enrollment data, should I just use that instead?

In [ ]:
# Define the cohort date range
#start_date = pd.Timestamp('2022-07-01') # UPDATE DATES for each cohort range
#end_date = pd.Timestamp('2023-06-30')

# Filter the DataFrame to keep rows within the date range
filtered_data = itss[(itss['Program Enrollment End Date'] >= cohort_start_date) &
                 (itss['Program Enrollment End Date'] <= cohort_end_date)]

filtered_data.shape

(5070, 38)

In [ ]:

## UPDATE 1/28/25:

# Convert end date field in enrollments to datetime
enrollments['Enrollment End Date'] = pd.to_datetime(enrollments['Enrollment End Date'])

# Remove programs still active as of end of the cohort/reporting period
filtered_enrollments = enrollments[(enrollments['Enrollment End Date'] >= cohort_start_date) &
                 (enrollments['Enrollment End Date'] <= cohort_end_date)]

print(f'Enrollments total: {enrollments.shape[0]}\nEnrollments filtered for exits during the reporting period: {filtered_enrollments.shape[0]}')

# Aggregate all rows by ETO number, getting a list of all enrolled programs by participant
enrollments_agg = filtered_enrollments.groupby("ETO Case Number")["Program Name"].unique().reset_index()
enrollments_agg["Enrolled Programs"] = enrollments_agg["Program Name"].apply(list)  # Ensure it's a list
# drop the "Program Name" Column to avoid confusion
enrollments_agg = enrollments_agg.drop(columns=["Program Name"])

print(f'Enrollments aggregated (unique participants): {enrollments_agg.shape[0]}')

Enrollments total: 5827
Enrollments filtered for exits during the reporting period: 3314
Enrollments aggregated (unique participants): 2553


In [ ]:
#enrollments_agg.to_csv('enrollments_check.csv', index=False) # Just checking, remove

In [ ]:
# Merging aggregated enrollments with aggregated itss data (aggregated just means the ETO numbers are unique in both datasets)
merged_itss_enrollments = filtered_data.merge(enrollments_agg[["ETO Case Number", "Enrolled Programs"]], on="ETO Case Number", how="left")

In [ ]:
merged_itss_enrollments.shape
# Should be same as filtered_data

(5070, 39)

In [ ]:
# From process doc:
# "Sort and delete based on field “Program of Enrollment” except:
    # WIOA Adult, WIOA Dislocated Worker, WIOA Out of School Youth, WIOA In School Youth.

# Filter for only those programs:
keep = ['WIOA Adult', 'WIOA Dislocated Worker', 'WIOA Out of School Youth', 'WIOA In School Youth']

# Define a function to check if any enrolled program is in "keep"
def check_wioa(enrolled_programs):
    if isinstance(enrolled_programs, list):  # Ensure it's a list before checking
        return any(program in keep for program in enrolled_programs)  # Check membership
    return False  # Default to False if NaN or not a list

# Apply the function to create the "WIOA Eligible" column
# This creates a boolean (true / false) column for "WIOA"
merged_itss_enrollments["WIOA"] = merged_itss_enrollments["Enrolled Programs"].apply(check_wioa)

# Keep only participants who are WIOA Eligible
clean_data = merged_itss_enrollments[merged_itss_enrollments["WIOA"]]


In [ ]:
clean_data

,Is Youth Program,Program Category,WDA Name,Office Name,Last Updated By,Created By,Date Entered,Date Created,Date Last Updated,Opted Out of Data Sharing?,...,Program Enrollment Status,Program Enrollment Start Date,Program Enrollment End Date,Local Program on Program Enrollment TP,Contract (if applicable) on Program Enrollment TP,ETO Office Number,O*NET Code_14584,Staff Restricted,Enrolled Programs,WIOA
0,Not Youth Program,Other Than WIA/WIOA/Trade Enrollments,WDA 04 - Snohomish,Worksource Everett,"Aphkas, Tarseny","Aphkas,Tarseny",11/20/20,11/20/20,6/10/21,No,...,Enrolled,2020-11-04,2021-01-14,NaN,WDA4-DWER,797,NaN,Yes,"[COVID-19 Employment Recovery DWG, WIOA Disloc...",True
2,Not Youth Program,Other Than WIA/WIOA/Trade Enrollments,WDA 04 - Snohomish,Worksource Everett,"Aphkas, Tarseny","Aphkas,Tarseny",12/29/20,12/29/20,12/29/20,No,...,Enrolled,2020-12-10,2021-08-17,NaN,NaN,797,17-2011.00,Yes,"[COVID-19 Employment Recovery DWG, Rapid Respo...",True
3,Not Youth Program,Other Than WIA/WIOA/Trade Enrollments,WDA 04 - Snohomish,Worksource Everett,"Aphkas, Tarseny","Aphkas,Tarseny",12/29/20,12/29/20,1/12/21,No,...,Enrolled,2020-12-22,2021-03-15,NaN,WDA4-RRI4,797,NaN,Yes,"[COVID-19 Employment Recovery DWG, Rapid Respo...",True
4,Not Youth Program,Other Than WIA/WIOA/Trade Enrollments,WDA 04 - Snohomish,Worksource Everett,"Aphkas, Tarseny","Aphkas,Tarseny",12/29/20,12/29/20,1/12/21,No,...,Enrolled,2020-12-22,2021-03-15,NaN,NaN,797,NaN,Yes,"[COVID-19 Employment Recovery DWG, Rapid Respo...",True
5,Not Youth Program,Other Than WIA/WIOA/Trade Enrollments,WDA 04 - Snohomish,Worksource Everett,"Aphkas, Tarseny","Aphkas,Tarseny",12/29/20,12/29/20,1/12/21,No,...,Enrolled,2020-12-22,2021-03-15,NaN,WDA4-RRI4,797,NaN,Yes,"[COVID-19 Employment Recovery DWG, Rapid Respo...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5065,Youth Program,WIA and WIOA Enrollments,WDA 04 - Snohomish,WorkSource Youth Center,"Wabnitz, Elijah","Wabnitz,Elijah",3/3/21,3/3/21,3/3/21,No,...,Enrolled,2021-01-22,2021-08-25,NaN,20-RWS-X-420-Youth,763,NaN,NaN,"[WIOA Out of School Youth, WIOA Adult]",True
5066,Youth Program,WIA and WIOA Enrollments,WDA 04 - Snohomish,WorkSource Youth Center,"Wabnitz, Elijah","Wabnitz,Elijah",3/5/21,3/5/21,3/9/21,No,...,Enrolled,2021-02-24,2021-11-04,NaN,20-RWS-X-420-Youth,763,NaN,NaN,"[WIOA Adult, WIOA Out of School Youth]",True
5067,Youth Program,WIA and WIOA Enrollments,WDA 04 - Snohomish,WorkSource Youth Center,"Wabnitz, Elijah","Wabnitz,Elijah",3/5/21,3/5/21,3/5/21,No,...,Enrolled,2021-03-05,2021-04-27,NaN,20-RWS-X-420-Youth,763,NaN,NaN,"[WIOA Out of School Youth, WIOA Adult]",True
5068,Youth Program,WIA and WIOA Enrollments,WDA 04 - Snohomish,WorkSource Youth Center,"Wabnitz, Elijah","Wabnitz,Elijah",3/9/21,3/9/21,3/9/21,No,...,Enrolled,2020-10-26,2021-06-10,NaN,20-RWS-X-420-Youth,763,NaN,NaN,"[WIOA Adult, WIOA Out of School Youth]",True


In [ ]:
# Replace blanks with "no value"
clean_data['Actual Outcome'] = clean_data['Actual Outcome'].fillna('No Value').replace('', 'No Value')
# NOTE: Local Services report optimized uses the field "Service Outcome". ITS report uses fied "Actual Outcome".
# But both reports should get you the same answer.

clean_data.shape

<ipython-input-98-24666719c1f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Actual Outcome'] = clean_data['Actual Outcome'].fillna('No Value').replace('', 'No Value')


(4288, 40)

In [ ]:
# list of approved services for denominator

services = [
'ADULT EDUCATION & LITERACY WITH TRAINING',
'ADULT EDUCATION &amp;amp; LITERACY WITH TRAINING',
'ADULT EDUCATION AND LITERACY WITH TRAINING',
'Adult Education and Literacy with Training (2.0)',
'Alternative Secondary School Services or Dropout Recovery Services (Youth Only)',
'Apprenticeship Training',
'ARRA 10% CLASS-SIZE CONTRACTED TRAINING',
'Class-Size Contracted Training',
'EDUCATIONAL ACHIEVEMENT SERVICES (YOUTH ONLY)',
'Educational Achievement Services (Youth Only)',
'ENGLISH AS A SECONDARY LANGUAGE',
'English as a Secondary Language (2.0)',
'Entrepreneurial Training',
'Entrepreneurial Training (2.0)',
'Increased Capacity Training',
'Occupational Skills Training',
'Occupational Skills Training (2.0)',
'Occupational Skills Training (Youth Only)',
'Pre-Requisite Training (TAA Only)',
'PURSUING GED/DIPLOMA/CERTIFICATE (YOUTH ONLY)',
'Short Term Classroom Training',
'SHORT TERM CLASSROOM TRAINING',
'SHORT-TERM CLASSROOM TRAINING',
'SKILLS UPGRADING AND RETRAINING',
'TAA Approved Training',
'TUTORING, STUDY SKILLS TRAINING, INSTRUCTION, AND DROPOUT PREVENTION (YOUTH ONLY)',
'Alternative Secondary School Services or Dropout Recovery Services (Youth Only)',
'Apprenticeship Training Entrepreneurial Training (2.0)',
'Increased Capacity Training',
'Incumbent Worker',
'Incumbent Worker Training', # I added this manually, not sure why it's not in the DOL PIRL fucntional instructions list...
'Workplace Training with Related Instruction Workplace Training with Related Instruction',
]



In [ ]:
# Apply the function after grouping by 'Actual Outcome'
# TODO - hold off on this for now, instructions unclear. ???
# final_data = clean_data.groupby('Actual Outcome', group_keys=False).apply(filter_duplicates) # this works but gives deprecation warning


# Apply the function with include_groups=False to exclude the grouping columns from the operation
#final_data = clean_data.groupby('Actual Outcome', group_keys=False, as_index=False).apply(filter_duplicates, include_groups=False)

# Dictionary mapping services to expected appropriate outcomes
outcome_mapping = {
    'Positive training outcome': ['Associates Degree Earned',
                     'Attained GED/HS Equivalency',
                     'Attained High School Diploma',
                     'Bachelors Degree Earned',
                     'Locally Authorized Credential Earned',
                     'Masters Or Doctorate Degree Earned',
                     'Occupational Skills Certificate Or Credential Earned',
                    ],
    'Incorrect training outcome': ['Not Required For This Service',
                                   'OJT Completed Not Hired',
                                   'OJT Hired By Non Training Related Employer',
                                   'OJT Hired by OJT Employer',
                                   'OJT Hired By Other Training Related Employer',
                                   'OJT Successful Completion',
                                   'Work Readiness Credential Earned',
                                   'Younger Youth Skill Goal Attained',
                                   'Younger Youth Skill Goal Not Attained',
                                   'No Value'
                                  ],
    'Expected training outcome': ['Associates Degree Earned',
                                  'Attained GED/HS Equivalency',
                                  'Attained High School Diploma',
                                  'Bachelors Degree Earned',
                                  'Locally Authorized Credential Earned',
                                  'Masters Or Doctorate Degree Earned',
                                  'Occupational Skills Certificate Or Credential Earned',
                                  'Completed-Certificate/Credential Pending',
                                  'Completed-No Credential/Certificate',
                                  'Did Not Complete',
                                  'Unknown Outcome - Historical Data Cleanup',
                                 ]
}

# Function to categorize based on service description and actual outcome
def categorize_service(row):
    service = row['Service Description']
    outcome = row['Actual Outcome'] # change Actual to Service and vv depending on the report used

    if service in services: # for each service in the qualifying training services list (see above)
        if outcome in outcome_mapping['Positive training outcome']: # and has a positive credential outcome
            if service == "Alternative Secondary School Services or Dropout Recovery Services (Youth Only)":
                return 'check for employment/post-secondary' # trigger manual check of employment (for now, automate later)
            return 'Numerator' # goes in the numerator
        if outcome in outcome_mapping['Incorrect training outcome']: # and has an incorrect outcome for a training service
            return 'Data entry error: training service requires training-related outcome' # trigger return to subs
        if outcome == "Completed-No Credential/Certificate":
            return 'Data check/possible error: credential or certificate expected for a completed training service'
        return 'Denominator' # service is a qualifying service but has negative outcome (not in positive outcome list) and not incorrect outcome

    else: # if service not in qualifying service list (i.e., all other services)
        if outcome in outcome_mapping['Positive training outcome']: # and has a positive training outcome
            if service == "Training Paid By Other":
                return 'Not considered in CA calculation'
            return 'Data entry error: positive training outcome for non-training service'
        if outcome == "Completed-Certificate/Credential Pending":
            if service == "Training Paid By Other":
                return 'Not considered in CA calculation'
            return 'Data entry error: pending credential for for non-training service'
        else: # service is not a qualifying training service and outcome is also not a positive training outcome
            return 'Not considered in CA calculation'


# Apply the function to create a new column categorizing the services
clean_data['CA screen result'] = clean_data.apply(categorize_service, axis=1)



clean_data.shape

<ipython-input-100-678430c8eb51>:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['CA screen result'] = clean_data.apply(categorize_service, axis=1)


(4288, 41)

In [ ]:
# Checking for duplicates

# Create a new column 'duplicates' and initialize it with 'No'
clean_data['duplicates'] = 'No'

# Group 1: Rows where "CA screen result" is NOT 'Not considered in CA calculation'
# This is also just the subset of every ITSS TP that does or may qualify as CA
group1 = clean_data[clean_data['CA screen result'] != 'Not considered in CA calculation']

# Group 2: Rows where "CA screen result" IS 'Not considered in CA calculation'
group2 = clean_data[clean_data['CA screen result'] == 'Not considered in CA calculation']

# Mark duplicates within Group 1 (based on ETO Case Number)
group1.loc[group1.duplicated(subset=['ETO Case Number'], keep=False), 'duplicates'] = 'Yes'

# Combine Group 1 and Group 2 back into one DataFrame
df_combined = pd.concat([group1, group2], ignore_index=True)

# Now df_combined contains the original data with duplicates flagged only for Group 1

ca_subset = group1




<ipython-input-101-d4fa9e3e13d0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['duplicates'] = 'No'


In [ ]:
#group1.to_csv('group1.csv', index=False)

In [ ]:
print(df_combined.shape)
ca_subset.shape

(4288, 42)


(199, 42)

## Merge Outcomes report for employment and secondary education

This section uses the filtered and cleaned data from the ITSS report, the output of all the cells above, and merges it with the outcomes report.

Outcomes report is from ETO: Local Reporter Data --> `Program Enrollment Outcomes`
For this report, you can use the same date range as the reporting period.

Again, save as csv and include the date ranges in the file name and save.



In [ ]:
# Cleaning column names for weird characters
outcomes.columns = outcomes.columns.str.replace(r'[^\x00-\x7F]+', ' ')  # Replace non-ASCII characters with space
outcomes.columns = outcomes.columns.str.strip()  # Strip leading/trailing whitespace

In [ ]:
outcomes.columns

Index(['WDA Name', 'Office Name', 'Seeker Name', 'Case Number', 'Is Opted Out',
       'Phone', 'Email', 'Preferred Method of Contact',
       'Unique Enrollment ID (can be linked to Program Enrollment Report)',
       'Program of Enrollment', 'Local Program Detail',
       'Contract (If applicable)',
       'Enrollment Start Date on Program Enrollment TP',
       'Enrollment End Date on Program Enrollment TP',
       'Outcome TP Originally Recorded By', 'Outcome TP Created on Behalf Of',
       'Outcome TP Last Updated By', 'Outcome TP Date Last Modifed',
       'Outcome TP Date Taken',
       'Household income greater than 200% of Federal Poverty Level',
       'Program Completion Date', 'Reason for Completion',
       'Unsubsidized Employment', 'Employed', 'Self-Employed',
       'Employment Start Date', 'Wage', 'Wage Frequency', 'Annualized Wage',
       'Occupational Code',
       'Employer Name (if staff selected from employer drop-down list)',
       'Employer Name (if staff did

In [ ]:
# Download outcomes report from ETO
# I used Local Reporter Data > Program Enrollment Outcomes
    # Set the start date to way back in April 2017, as with the TP report, since this report uses enrollment dates to pull
    # Same end date as cohort.

# Cleaning column names for merging
outcomes.rename(columns={"Case Number": "ETO Case Number"}, inplace=True)
outcomes.rename(columns={"Program of Enrollment": "Program Of Enrollment_outcomes"}, inplace=True)



In [ ]:
# ADDING A STEP, need to aggregate outcomes before merging.]

# Step 1: Define a function to aggregate values into lists
def aggregate_to_list(series):
    return series.tolist()  # Converts each group into a list (including NaNs)

# Step 2: Aggregate outcomes dataset, keeping all values in lists
outcomes_agg = outcomes.groupby("ETO Case Number").agg({
    "Unsubsidized Employment": aggregate_to_list,
    "Reason for Completion": aggregate_to_list
}).reset_index()

# Perform an inner join to keep only rows that match between df_filtered and outcomes
merged_data = pd.merge(
    ca_subset,
    outcomes_agg,
    on=['ETO Case Number'],  # Use both columns for accurate matching
    how='left',
    suffixes=('', '_outcomes')  # Adds '_outcomes' suffix to duplicated columns from 'outcomes'
)

print(f"{merged_data.shape=}")

merged_data.shape=(199, 44)


In [ ]:
merged_data.columns

Index(['Is Youth Program', 'Program Category', 'WDA Name', 'Office Name',
       'Last Updated By', 'Created By', 'Date Entered', 'Date Created',
       'Date Last Updated', 'Opted Out of Data Sharing?', 'Name',
       'ETO Case Number', 'Unique Record ID',
       'Service Participation Episode ID', 'Service Description',
       'Method of Contact', 'Projected activity start date',
       'Projected activity end date', 'Activity Start Date',
       'Activity End Date', 'Actual Outcome', 'Contract',
       'Contract (if not listed above)', 'Training Provider',
       'Provider Name (if not listed above)', 'Course', 'Other Program',
       'Service is Associated with a Program Enrollment TP?',
       'Unique Enrollment ID', 'Program Of Enrollment',
       'Program Enrollment Status', 'Program Enrollment Start Date',
       'Program Enrollment End Date', 'Local Program on Program Enrollment TP',
       'Contract (if applicable) on Program Enrollment TP',
       'ETO Office Number', 'O*NET

In [ ]:
# Create a new column for program of enrollment that accounts for the duplicates created in the merge (for coenrollments)
# Some people not in the outcomes data will need to have the value filled in from Program of Enrollment on the TP
# Should be relatively few, and should we flag them somehow? They should have an outcome if they were exited.
    # Data entry correction?

# Create 'Merged Programs' by filling blanks in 'Program of Enrollment2' with values from 'Program of Enrollment'
#merged_data['Merged Programs'] = merged_data['Program Of Enrollment_outcomes'].fillna(merged_data['Program Of Enrollment'])

In [ ]:
# Update the CA screen result with outcome data for employment or post-secondary, only for those who attained GED/HS

# TODO I should skip the previous step of creating the placeholder and just use the "actual outcome + service description" for this.

# Step 0: Create a copy of 'CA screen result' to retain the original flags
merged_data['CA screen result_original'] = merged_data['CA screen result']
merged_data['CA screen result_updated'] = merged_data['CA screen result']

merged_data.loc[
    (merged_data["CA screen result"] == "check for employment/post-secondary") &
    (merged_data["Unsubsidized Employment"].str.contains("Yes", regex=False)),
    "CA screen result_updated"
] = "Numerator"

merged_data.loc[
    (merged_data["CA screen result"] == "check for employment/post-secondary") &
    (merged_data["Reason for Completion"].str.contains("Entered a Post Secondary Education", regex=False)),
    "CA screen result_updated"
] = "Numerator"



# Step 2: Update remaining 'check for employment/post-secondary' entries to 'CA Denominator'
merged_data.loc[merged_data['CA screen result_updated'] == 'check for employment/post-secondary', 'CA screen result_updated'] = 'Denominator'

In [ ]:
# Dealing with duplicates

# For the rows where "duplicates" is "yes"
    # Sort or group by paired ETO Case Numbers, and check for each pair (or group) of same ETO Case Numbers:
        # If "Service Description" is the same for both duplicates,
            # check if there is a positive outcome, and count that one
            # mark the duplicated (not counted) row as "Duplicate training serivce - possible data entry error"
        # If different service
            # Check for positive/negative outcomes, count positive one
                # If both outcomes are positive, chose one, preferring one that is not Alternative High School.


def deduplicate_CA_screen(group):
    """
    Processes duplicate CA screen results per ETO Case Number.
    Ensures each participant is counted only once based on specified rules.
    """

    # If there's only one row in the group, keep it as is and exit early
    if len(group) == 1:
        group["CA final"] = group["CA screen result_updated"]
        return group  # Exit early to prevent any unnecessary processing

    # Identify key conditions
    has_numerator = "Numerator" in group["CA screen result_updated"].values
    has_denominator = "Denominator" in group["CA screen result_updated"].values
    has_data_entry_error = group["CA screen result_updated"].str.contains("Data", na=False).any()

    # Identify rows
    numerator_rows = group[group["CA screen result_updated"] == "Numerator"]
    denominator_rows = group[group["CA screen result_updated"] == "Denominator"]
    data_error_rows = group[group["CA screen result_updated"].str.contains("Data", na=False)]

    ### Handle when both Numerator & Denominator exist in group ###
    if has_numerator and has_denominator:
        # Keep Numerator(s) as is
        group.loc[numerator_rows.index, "CA final"] = "Numerator"
        # Change Denominator(s) to "negative outcome canceled out by Numerator - delete"
        group.loc[denominator_rows.index, "CA final"] = "negative outcome canceled out by Numerator - delete"

    ### FIX: Ensure "check for employment/post-secondary" rows are preserved ###
    group.loc[group["CA screen result_updated"] == "check for employment/post-secondary", "CA final"] = "check for employment/post-secondary"

    ### CASE 1: Data Entry Errors Handling ###
    if has_data_entry_error:
        if has_numerator:
            group.loc[data_error_rows.index, "CA final"] = "data entry issue negated by other positive outcome - delete"
        else:
            group.loc[data_error_rows.index[0], "CA final"] = group.loc[data_error_rows.index[0], "CA screen result"]
            if len(data_error_rows) > 1:
                group.loc[data_error_rows.index[1:], "CA final"] = "duplicate - delete"

    ### CASE 2: Single Service Type ###
    if group["Service Description"].nunique() == 1:
        if has_numerator:
            group.loc[numerator_rows.index[0], "CA final"] = "Numerator"
            group.loc[numerator_rows.index[1:], "CA final"] = "Duplicate training service - delete"
        else:
            group.loc[group.index[0], "CA final"] = "Denominator"
            group.loc[group.index[1:], "CA final"] = "Duplicate training service - delete"

    ### CASE 3: Multiple Service Types ###
    elif has_numerator:
        if len(numerator_rows) == 1:
            group.loc[numerator_rows.index[0], "CA final"] = "Numerator"
        else:
            non_alt_hs_rows = numerator_rows[numerator_rows["Service Description"] != "Alternative Secondary School Services or Dropout Recovery Services (Youth Only)"]
            preferred_index = non_alt_hs_rows.index[0] if not non_alt_hs_rows.empty else numerator_rows.index[0]
            group.loc[preferred_index, "CA final"] = "Numerator"
            group.loc[numerator_rows.index.difference([preferred_index]), "CA final"] = "More than one qualifying service - delete"

    ### DEFAULT: Ensure all other rows retain original values ###
    group.loc[:, "CA final"] = group["CA final"].fillna(group["CA screen result_updated"])

    return group

# Apply deduplication process
merged_data["CA final"] = None  # Initialize column
merged_data = merged_data.groupby("ETO Case Number", group_keys=False).apply(deduplicate_CA_screen)

# One more step, to check for short dates on any qualifying trainings:



<ipython-input-111-6a79dc7bd6ce>:79: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_data = merged_data.groupby("ETO Case Number", group_keys=False).apply(deduplicate_CA_screen)


In [ ]:
print(ca_subset.shape)
merged_data.to_csv('duplicates_flag.csv', index=False)

(199, 42)


In [ ]:
# Now delete all the flagged rows.
# Keep only rows that do NOT contain "delete" in "CA screen result_updated"
ca_subset_clean = merged_data[~merged_data["CA final"].str.contains("delete", na=False)]


In [ ]:
ca_subset_clean.shape

(166, 47)

In [ ]:
duplicates = ca_subset_clean["ETO Case Number"].duplicated().sum()
print(f"Number of duplicate ETO Case Numbers: {duplicates}")

# Any duplicates should be explained by data entry issues/checking for employment ONLY.

Number of duplicate ETO Case Numbers: 3


In [ ]:
#display(ca_subset_clean)

In [ ]:
# UPDATE, 1/28/25
# Step 1: Explode "Enrolled Programs" so each program gets its own row
expanded_df = ca_subset_clean.explode("Enrolled Programs")

# Step 2: Count "CA screen result_updated" by program
program_counts = expanded_df.groupby(["Enrolled Programs", "CA final"]).size().unstack(fill_value=0)


In [ ]:
display(program_counts) #accounts for coenrollment

CA final,Data check/possible error: credential or certificate expected for a completed training service,Data entry error: pending credential for for non-training service,Data entry error: positive training outcome for non-training service,Data entry error: training service requires training-related outcome,Denominator,Numerator
Enrolled Programs,,,,,,
COVID-19 Disaster Recovery DWG,0,0,2,1,2,3
COVID-19 Employment Recovery DWG,1,0,1,4,7,6
CPP - Claimant Placement Program,0,0,0,0,1,0
Career Connect WA,0,0,0,0,1,0
JMI - Job Match Initiative,0,0,0,0,1,0
Opioid Demonstration DWG PY18-PY19,0,0,2,1,6,3
Rapid Response Increased Employment Initiative (RRIE),0,0,1,5,6,3
Trade Assistance Act,3,0,0,3,41,20
WIOA Adult,2,0,4,5,40,24


In [ ]:
# Combine youth progams
program_counts.loc["Youth"] = program_counts.loc[["WIOA In School Youth", "WIOA Out of School Youth"]].sum()

# Drop the original youth categories
program_counts.drop(index=["WIOA In School Youth", "WIOA Out of School Youth"], errors="ignore", inplace=True)

# Keep only rows where "Enrolled Programs" is in the "programs" list - which is actually called "keep"
# Filter for only relevant programs (including "Youth")
program_counts = program_counts.loc[program_counts.index.isin(keep + ["Youth"])]

# Step 2: Create "Total Denominator" column (Numerator + Denominator)
program_counts["Total Denominator"] = program_counts.get("Numerator", 0) + program_counts.get("Denominator", 0)

# Step 3: Compute percentage (Numerator / Total Denominator * 100)
program_counts["Percentage"] = (program_counts["Numerator"] / program_counts["Total Denominator"]) * 100

# Step 4: Replace NaN values in "Percentage" with 0 (in case of division by zero)
#program_counts["Percentage"] = program_counts["Percentage"].fillna(0, inplace=True)

display(program_counts)

<ipython-input-119-78acab56909d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  program_counts["Total Denominator"] = program_counts.get("Numerator", 0) + program_counts.get("Denominator", 0)
<ipython-input-119-78acab56909d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  program_counts["Percentage"] = (program_counts["Numerator"] / program_counts["Total Denominator"]) * 100


CA final,Data check/possible error: credential or certificate expected for a completed training service,Data entry error: pending credential for for non-training service,Data entry error: positive training outcome for non-training service,Data entry error: training service requires training-related outcome,Denominator,Numerator,Total Denominator,Percentage
Enrolled Programs,,,,,,,,
WIOA Adult,2,0,4,5,40,24,64,37.500000
WIOA Dislocated Worker,5,1,3,7,53,28,81,34.567901
Youth,0,0,0,3,41,5,46,10.869565


In [ ]:
program_counts.to_csv(f'{shared_drive_path}Program Counts_{report_date}_{quarter}.csv', index=True)

In [ ]:
#regular_counts = ca_subset_clean.groupby(["Program Of Enrollment", "CA final"]).size().unstack(fill_value=0)

In [ ]:
#display(regular_counts) # Does not consider coenrollment, only looks at "Program of Enrollment" on qualifying CA TP

In [ ]:
merged_filename = f'{shared_drive_path}Credential_Attainment_{report_date}_{quarter}.csv'

ca_subset_clean.to_csv(merged_filename, index=False)

In [ ]:
# Export for manual checks

manual_check_filename = f'{shared_drive_path}credentials_check_{report_date}_{quarter}.csv'
merged_data.to_csv(manual_check_filename, index=False)

In [ ]:
# TODO: Follow up services